# Preamble 

This lab uses the climlab package https://climlab.readthedocs.io/en/latest/ by Brian Rose at the Unviersity of Albany.

This follows on closely from lab 3, but with a new function that you can use.

The function getEquilRadConvColumn will setup and run the model for your, you can see the default arguements that it takes and modify them accordingly when you call the function.

The function makePlotsFromExperimentList will make some plots of the vertical profiles of temperature, moisture, radiative fluxes and heating rates, given a list of experiments. 

The function getRadDiagnostic will take in input values of surface temperature, atmospheric temperature, and moisture, and will rerun the radiation code and output the radiation diagnostics.

Some examples of how to use these two functions are shown below.

In [1]:
import climlab
import matplotlib.pyplot as plt
import numpy as np
from itertools import product
import scipy as sp
import xarray as xr

In [2]:
def getEquilRadConvColumn(
        return_model=False,
        num_lev=20,
        relHum=0.8,
        lapseRate=6,
        integrate_years=5,
        water_depth=5.,
        do_conv=True,
        albedo=0.3,
        CO2=0.000348,
        CH4=1.65e-06,
        N2O=3.06e-07,
        O2=0.21,
        CFC11=0.0,
        CFC12=0.0,
        CFC22=0.0,
        CCL4=0.0,
        O3_profile=np.array([5.38853507e-06, 9.86362297e-07, 3.46334801e-07, 1.90806332e-07,
                                    1.19700066e-07, 7.69083554e-08, 5.97316411e-08, 5.27011190e-08,
                                    4.80406196e-08, 4.44967931e-08, 4.18202246e-08, 3.99595858e-08,
                                    3.83838549e-08, 3.66179869e-08, 3.42885526e-08, 3.18505117e-08,
                                    2.93003951e-08, 2.69906527e-08, 2.49122466e-08, 2.28798533e-08]),
        O3fac=1.0,
        insolation=341.,
        cldfrac=0., 
        clwp=0., 
        ciwp = 0.,
        r_liq = 0.,
        r_ice = 0.,
        ):
    
    '''
    Mostly used as an interface to make it easy to 
    remember what the default GHGs are and change them
    '''

    # get the different components of the model
    # state - container for the vertical temperature profile
    # water vapour - set moisture based on relative humidity profile        
    # convection - hard convective adjustment
    # radiation - RRTM spectral radiation 

    state = climlab.column_state(num_lev=num_lev,
                                 num_lat=1,
                                 water_depth=water_depth)

    water_vapor = climlab.radiation.water_vapor.ManabeWaterVapor(state=state,
                                                                        name='Constant Relative Humidity')

    # water_vapor = climlab.radiation.water_vapor.FixedRelativeHumidity(state=state,
    #                                                                     relative_humidity=relHum,
    #                                                                     name='Constant Relative Humidity')
    
    state['q'] = water_vapor.q

    if do_conv:
        convection = climlab.convection.ConvectiveAdjustment(name='Convective Adjustment',
                                                  state=state,
                                                  adj_lapse_rate=lapseRate)    

    # create the ghg dict
    ghgDict = {'CO2': CO2,
                'CH4': CH4,
                'N2O': N2O,
                'O2': O2,
                'CFC11': CFC11,
                'CFC12': CFC12,
                'CFC22': CFC22,
                'CCL4': CCL4,
                'O3': O3fac*O3_profile}

    radiation = climlab.radiation.CAM3(name='Radiation',
                             state=state,
                             specific_humidity=water_vapor.q,
                             absorber_vmr=ghgDict,
                             cldfrac=cldfrac,
                             clwp=clwp,
                             ciwp=ciwp,
                             r_liq=r_liq,
                             r_ice=r_ice,
                             insolation=insolation,
                             return_spectral_olr=True,
                             albedo=albedo)

    # combine the components of the model together 
    if do_conv:
        model = climlab.couple([radiation,convection,water_vapor], name='RadConv')
    else:
        model = climlab.couple([radiation,water_vapor], name='RadOnly')

    # actually run the model 
    model.integrate_converge()
    model.compute_diagnostics()
    # print(model.OLR_spectral)

    if return_model:
        return model
    else:
        return model.to_xarray(diagnostics=True)

In [3]:
def getRadDiagnostic(
        input_state_Ts,
        input_state_Tatm,
        input_state_q,
        CO2=0.000348,
        CH4=1.65e-06,
        N2O=3.06e-07,
        O2=0.21,
        CFC11=0.0,
        CFC12=0.0,
        CFC22=0.0,
        CCL4=0.0,
        O3_profile=np.array([5.38853507e-06, 9.86362297e-07, 3.46334801e-07, 1.90806332e-07,
                                    1.19700066e-07, 7.69083554e-08, 5.97316411e-08, 5.27011190e-08,
                                    4.80406196e-08, 4.44967931e-08, 4.18202246e-08, 3.99595858e-08,
                                    3.83838549e-08, 3.66179869e-08, 3.42885526e-08, 3.18505117e-08,
                                    2.93003951e-08, 2.69906527e-08, 2.49122466e-08, 2.28798533e-08]),
        O3fac=1.0,
        insolation=341.,
        cldfrac=0., 
        clwp=0., 
        ciwp = 0.,
        r_liq = 0.,
        r_ice = 0.,
        albedo=0.1,
        ):

    # get the different components of the model
    # state - container for the vertical temperature profile
    # water vapour - set moisture based on relative humidity profile        
    # radiation - RRTM spectral radiation 

    # create the ghg dict
    ghgDict = {'CO2': CO2,
                'CH4': CH4,
                'N2O': N2O,
                'O2': O2,
                'CFC11': CFC11,
                'CFC12': CFC12,
                'CFC22': CFC22,
                'CCL4': CCL4,
                'O3': O3fac*O3_profile}

    # this is necessary because we output the states to xarray,
    # so have to convert back to state.    
    state = climlab.column_state(num_lev=len(input_state_Tatm), num_lat=1)
    state['Ts'][:] = input_state_Ts
    state['Tatm'][:] = input_state_Tatm

    radiation = climlab.radiation.CAM3(name='Radiation',
                             state=state,
                             specific_humidity=input_state_q,
                             absorber_vmr=ghgDict,
                             cldfrac=cldfrac,
                             clwp=clwp,
                             ciwp=ciwp,
                             r_liq=r_liq,
                             r_ice=r_ice,
                             insolation=insolation,
                             albedo=albedo)
    

    radiation.compute_diagnostics()

    # output the data as an xarray dataset
    return radiation.to_xarray(diagnostics=True)



In [4]:
def makePlotsFromExperimentList(rl,labell,
colorlist=['black','blue','red','green','orange','purple'],
plotDiff=False,
outputname=None,
suptitle='TITLE GOES HERE'
):

    titles=['T (K)','q(kg/kg)','','SW Flux (W/m^2)','LW Flux (W/m^2)','Heating Rates (K/day)']

    fig,axs=plt.subplots(2,3,figsize=(12,10),dpi=100)

    fig.delaxes(axs[0,-1])

    nl=range(0,len(rl))

    for n,r,l,c in zip(nl,rl,labell,colorlist):

        if plotDiff:
            r=r-rl[0]

        # temperature
        axs[0,0].plot(r.Tatm,r.lev,
                    label=l,color=c)

        # moisture
        axs[0,1].plot(r.q,r.lev,
                      label=l,color=c)

        if n==0:
            label0='FluxUp'
            label1='FluxDown'
            label2='FluxNet'
        else:
            label0=None
            label1=None
            label2=None
 
        # shortwave
        axs[1,0].plot(r.SW_flux_up,r.lev_bounds,
                      label=label0,color=c,linestyle='--')
        axs[1,0].plot(r.SW_flux_down,r.lev_bounds,
                      label=label1,color=c,linestyle='-.')
        axs[1,0].plot(r.SW_flux_net,r.lev_bounds,
                      label=label2,color=c,linestyle='-')
        
        # longwave
        axs[1,1].plot(r.LW_flux_up,r.lev_bounds,
                      label=label0,color=c,linestyle='--')
        axs[1,1].plot(r.LW_flux_down,r.lev_bounds,
                      label=label1,color=c,linestyle='-.')
        axs[1,1].plot(r.LW_flux_net,r.lev_bounds,
                      label=label2,color=c,linestyle='-')
        
        if n==0:
            label0='SW'
            label1='LW'
        else:
            label0=None
            label1=None

        # heating rates
        axs[1,2].plot(r.TdotSW,r.lev,
                        color=c,label=label0,linestyle='--')
        axs[1,2].plot(r.TdotLW,r.lev,
                        color=c,label=label1,linestyle='-.')

    for i,j in product(range(0,2),range(0,3)):
        axs[i,j].set_ylim([1e3,0])
        axs[i,j].set_title(titles[3*i+j])
        axs[i,j].set_ylabel('p (hPa)')

    axs[0,0].legend()
    axs[1,0].legend()
    axs[1,-1].legend()

    fig.suptitle(suptitle)
    fig.tight_layout()

    if not outputname is None:
        plt.savefig(outputname)

    plt.show()

# Example 3 - Calculating radiative diagnostics

In [5]:
# compute the model solutions for a radiative column 
ds=getEquilRadConvColumn()

Total elapsed time is 7.994695027025902 years.


In [6]:
# note - for some reason climlab needs to get numpy arrays and not datasets to initialize
# if you try passing the data in as xarray as xarray objects it will throw an error at you.
ds_rad = getRadDiagnostic(ds.Ts.data, ds.Tatm.data, ds.q.data)

# lets check and see that the diagnostics worked
# since we passed the profiles from the equillibrium state back to the diagnostic function it should give us 0.
print(ds_rad.OLR.data - ds.OLR.data)

[0.]



# Questions {-}

## 1. Perturbing the model {-}

For the next two sub-questions we are going to consider perturbing the model in the following ways:

1. doubling the CO2
2. doubling the O3
3. doubling the CH4
4. halfing the albedo 

### 1.1 perturbing in the radiative only model {-}

- run each perturbation with the convection off and make a plot.

- What does each perturbation do to the vertical structure of the temperature perturbation, and why?

- What is different about the albedo simulation compared to the others? (hint: look at the SW and LW plots).

### 1.2 perturbing in the radiative-convective model {-}

- run each perturbation with the convection on and make a plot.

- How does the vertical structure of the temperature perturbations compare to the model without convection?

### 2 CO2 doubling while variyng the lapse rate {-}

- run the doubled CO2 expeirment but now while varying the lapse rate between 3, 6, and 9 K/km.

- What is happening to the vertical structure of convection under the CO2 doubling, and how is this affected by the change in lapse rate?

- how does the vertical structure of the temperature and humidity responses to CO2 doubling vary with lapse rate? 

### 3 Making your own Radiative Kernel (sort of) {-}

- Run the model to equillibrium with a lapse rate of 3K/km (or use the data from a pervious run). Using the function getEquilRadConvColumn(), change the air temperature by 1K in each layer from the lowest layer to 200 hPa and calculate the new OLR. Take the difference between OLR calculated with and without the perturbation to find the temperature kernel at that level $$ K^p_T = OLR(T_a(p)+1K) - OLR(T_a(p)) $$

- Repeat the above calculations but with a lapse rate of 9K/km.

- Plot $K_T$ for both calculations. Why do you think that there is a difference in the upper troposphere?